<a href="https://colab.research.google.com/github/MWSteams/4590FSP/blob/master/Chemicals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Chemicals

Authors: Bridget Childs, Ananya Gangadhar, Jacqueline Wong

[/Monroe/]: # (Why did you delete all of my comments? I tried to be as emphatic as possible that I wanted to see your responses to my comments. From the history on Github I see that Bridget deleted all of comments 8 days ago. I suspect this happened because you started working on an older version of the file that you had on your computer rather than using the version from Github. My review isn't worth much if you delete it.)

[/Monroe/]: # (You are making reasonable progress. Fix your estimates of flow rates and volumes, design the plumbing system, specify the pumps required, review your report for spelling errors [too bad that spell checkers don't work yet], design the storage tanks, etc. )



In [1]:
!pip install aguaclara
!pip install molmass

     |████████████████████████████████| 92kB 2.4MB/s 
     |████████████████████████████████| 163kB 8.1MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.2.0-cp36-none-any.whl size=99623 sha256=c235e7895d4fa7a01b9ac9d0522189214884ac1906b2e66782462626ba5fbc4c
  Stored in directory: /root/.cache/pip/wheels/34/e9/34/0d3722243083a982623290c205c14a08b54ed290ee29219fd2
  Created wheel for pint: filename=Pint-0.8.1-py2.py3-none-any.whl size=134156 sha256=4756b61225c0350f7e1d7dc1447249c9aff1ab1731ee286eeb8fe022bc21c3b8
  Stored in directory: /root/.cache/pip/wheels/b4/56/36/7c4d6a4e3660e233b95e3b0482551915cb184abca0f93ee1dd
Successfully built aguaclara pint
     |████████████████████████████████| 71kB 2.1MB/s 


In [2]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import molmass as mol
!wget https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
import Fairmont as fmt

--2020-04-30 13:56:54--  https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2666 (2.6K) [text/plain]
Saving to: ‘Fairmont.py’

Fairmont.py         100%[===================>]   2.60K  --.-KB/s    in 0s      

2020-04-30 13:56:54 (46.1 MB/s) - ‘Fairmont.py’ saved [2666/2666]



# Task Definition

The Chemicals team will be in charge of designing the chemical dosage system. This will include determining the coagulant and polymer dosage, conducting a cost analysis based on anticipated dosages and vendors, and designing chemical storage. These tasks require a determination of the chemical feeds, feed hydraulics and plumbing, and available vendors. Additionally, our storage system design will include the number of storage tanks, tank size, location, footprint, and safety add-ons in order to meet OSHA requirements (e.g. spillover capture, eye washes, safety showers). Information will be primarily sourced from government regulations, Fairmont Treatment plant SOW, relevant vendors, and similar go-by systems.  

# Executive Summary

(Highlight dominant constraints and your recommendation for the design.)

# Constraints

* Design flow of the plant is 720 cfs.
* Effluent aresenic levels should be under 10 ug/L.
* Effluent TOC levels should be under 3 mg/L.
* Effluent turbidity should be under 3 NTU.
* Chemicals will be stored in an open tank farm and have at least a 30 day capacity  (Scope of work, pg 18).
* Proposed tank farm should have area under 200'x100'
* All hazardous chemicals must be stored in an area that is designed to contain spills ([OSHA regulations](https://www.safetyinfo.com/safe-chemical-storage-osha-requirements-free-index/)).
* All design values should be justifiable using design equations.
* The chemical feed system should be easy to understand and operate, and should have relatively low maintenance.
* The designed system should have a feedback loop wherein the chemical dosage is adjusted accordingly with a change in the plant flow rate.

**CEE 4590 Logistical constraints**
* Information in Scope of Work is often missing citations, and we are unsure of the source and degree of certainty of parameters like expected FeCl3 dosage.
* At present, we have no communication with the LA engineers on this project, only the Stantec engineers Mickey and Tori.
* It will be challenging to procure cost estimates from vendors since they are typically reluctant to share such information with students.
* It may be wise to add more redundancy to the chemical storage system than the recommended 30 days for situations like a shutdown due to a pandemic when supply chains are disrupted.. 

# Physics*

Since the Chemicals Team task concerns mainly the storage and delivery of coagulant and polymers, it is important to consider the energy and space that these chemicals will occupy. The **Energy equation** accounts the energy losses in fluid mechanics. Below is the general form, which will be useful as it is tailored to the chemical dosage design. [(AguaClara Textbook)](https://aguaclara.github.io/Textbook/Review/Review_Fluid_Mechanics.html#equation-review-review-fluid-mechanics-4)

$$\frac{p_{1}}{\rho g} + z_{1} + \alpha_{1} \frac{\bar v_{1}^2}{2g} + h_P = \frac{p_{2}}{\rho g} + z_{2} + {\alpha_{2}} \frac{\bar v_{2}^2}{2g} + h_T + h_L$$

Where:

p = pressure

ρ = density 

z = height 

v = velocity 

hp, ht = viscueous frictional loss due to pumps, turbines

hl = head loss


Head loss comprises of major losses from friction and minor losses from flow expansions, elbows. Major and minor loss equations are discussed below. 

 $$h_L = h_{\rm{f}} + h_e$$

Minor losses may be summarized as the equation below. K will vary depending on the minor loss (e.g. expansion, elbow, etc.). 

$$h_e = \sum K \frac{\bar v_{out}^2}{2g} =\left( \frac{A_{out}}{A_{in}} -1 \right)^2 \frac{\bar v_{out}^2}{2g}$$

The **Darcy-Weisbach equation** solves for major losses in the head loss equation. For this equation, it will be important to know the length and diameter of the pipes in our delivery system. Equation found in the [AguaClara Textbook](https://aguaclara.github.io/Textbook/Review/Review_Fluid_Mechanics.html#equation-darcy-weisbach). Additionally, there is an accompanying python function in the AguaClara python package to calculate major losses.

$$h_{\rm{f}} \, = \, {\rm{f}} \frac{L}{D} \frac{\bar v^2}{2g}$$

$$h_{\rm{f}} \, = \,{\rm{f}} \frac{8}{g \pi^2} \frac{LQ^2}{D^5}$$

Such that: 

hf = major loss 

f = Darcy friction factor
 
L = pipe length

Q = pipe flow rate
 
D = pipe diameter

AC Python Function = pc.headloss_fric(FlowRate, Diam, Length, Nu, PipeRough)

Below is the **Swamee-Jain equation** found from the [AguaClara Textbook](https://aguaclara.github.io/Textbook/Review/Review_Fluid_Mechanics.html#equation-swamee-jain). The Swamee-Jain equation helps determine the friction factor used in the Darcy-Weisbach equation. Ultimately, this will help calculate the major frictional losses of the system. 

$${\rm{f}} = \frac{0.25} {\left[ \log \left( \frac{\epsilon }{3.7D} + \frac{5.74}{{\rm Re}^{0.9}} \right) \right]^2}$$

Such that: 

f = friction factor 

ϵ = pipe roughness

D = pipe diameter

AC Python Function = pc.fric(FlowRate, Diam, Nu, PipeRough)

Other types of energy that may be of use to our chemical dosage system will be kinetic and potential energy, listed below. These are more general, but may be important to consider, especially with respect to the site's geography and elevation.   

$$ E_{potential} = mgh \,\,\,\,\, E_{kinetic} = mv^2/2$$

Ultimately, whatever power is neeeded to run the chemical dosage system will be supplied by pumps. For pumps, it will be important to consider how much power they will use. Below is the equation for power. E represents energy and t represents time. 

$$ P = \frac{E}{t} $$

Lastly, mass conservation will important to consider during our calculations. This will help us better understand how much storage will be required when considering the required dosage. 

$$Q_{in}C_{in} = Q_{out}C_{out}$$

# Table listing potential strategies to meet the constraints and treatment objectives*

| strategy | advantages  | disadvantages | data needed |
|---|---|---|---|
| open tank farm vs. enclosed storage | go-by, less expensive | less protection | weathering risk |
| 3 storage tanks vs. using even more | less complexity | lower safety factor | storage area |
| choosing vendor before choosing tank size | easier to pick model | more restricted | vendor details |
| 30-day storage vs. keeping even more | resilience (e.g. COVID) | more expensive | area, budget |
| single vs. multiple dosage system per train | maintenance, operator ease | fewer options if pipe fails | clogging risk |



# Vendors and contact information file

See this [tab delimited spreadsheet](https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt).

Add rows to the vendor spreadsheet as you find them. The sheet can easily be edited in Excel. Just make sure to save it as a tab delimited sheet. Push your changes to github so the entire class has access to the document.



In [3]:
vendors = pd.read_csv('https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt',sep='\t')
vendors.head()

,item description,vendor,contact first name,contact last name,vendor website,product info,date of contact,who made the contact,notes
0,plate settlers,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/literaturemedia/...,4/16/2020,Ian Cullings,NaN
1,sludge removal system,MRI,still,pending,http://www.meurerresearch.com/,http://www.meurerresearch.com/products/hoseles...,4/16/2020,Ian Cullings,NaN
2,plate settlers,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,https://www.monroeenvironmental.com/water-and-...,4/16/2020,Ian Cullings,NaN
3,sludge removal system,Monore Environmental,still,pending,https://www.monroeenvironmental.com/,http://www.monroeenvironmental.com/water-and-w...,4/16/2020,Ian Cullings,NaN
4,plate settler with hopper,Parkson,still,pending,https://www.parkson.com/,https://www.parkson.com/products/lamella-ecoflow,4/16/2020,Ian Cullings,Includes sloped sludge hopper in design


# Shared Assumptions

See this [python file, Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py)

Use this file to hold parameters that are in the SOW or elsewhere that
influence the design of the entire plant. This provides an easy mechanism
for all of us to share the same assumptions. Read the SOW to find sections that relate to your part of the design and enter all pertinent variables into [Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py) and then push those changes to github.

# Major Decisions

(Write several paragraphs describing the major decisions that you will need to make in order to create a design and outline a strategy for how you will obtain the necessary information to make those decisions.)

**Chemical Types**

The Fairmont Sedimentation Plant Scope of Work Document notes that two types of products will be used: a coagulant (specifically, ferric chloride) and emulsion polymer. Due to communication constraints with engineers at the Los Angeles Department of Water and Power, we are unlikely to fully understand the rationale behind the selection of ferric chloride as the coagulant. However, consider that it was already specified, we have decided to move forward with ferric chloride as the coagulant of choice.

On the other hand, the type of emulsion polymer was not specified in the Scope of Work document. There are many types and blends of polymers that are available in the water treatment market, many of which are proprietary and are vague in chemical makeup. 

**We will probably not be able to specify an emulsion polymer. However, we do know that the polymer is going to be cationic. We will be doing research to try to estimate the polymer density.**

It was noted by other teams that a high coagulant dosage could significantly alter the pH of the water. Since the Scope of Work did not specify the need for other chemical products, we made the assumption that the acid neutralizing capacity of raw water entering the Fairmont Sedimentatino Plant is adequate, and that additional chemicals beyond the coagulant and emulsion polymer are not required.

**Chemical Storage**

(TBC)

# Next Steps and Challenges

(Summarize the next steps that you will be taking to move the design forward. Identify any challenges where you may need assistance.)

1. Research cationic emulsion polymers and estimate density to calculate feed rate.
2. Research positive displacement diaphragm pumps and liquid feeder (valves, etc.)
3. Figure out tank configurations using LAAFP as reference to design storage area.
4. Update list of vendors for tanks and chemicals.

# Design

Begin writing the code to start your design. Define flow rates, mass rates, energy requirements, and any readily calculated dimensions, velocities, flows, etc.

[/Monroe/]: # (Begin by writing several paragraphs with equations explaining your design process. Then do the python analysis.)

[/Monroe/]: # (The tank volume calculation is incorrect. The density_fecl3 is NOT the concentration of the FeCl_3. You have to figure out what the 40% solution means and then turn that into the concentration of the FeCl3 stock solution. )

[/Monroe/]: # (Your design process for the pipe size and pump power is not robust. The goal is to develop design processes that work reliably even if the basis of design changes. The pipe diameter for the chemical feed system needs to change automatically if the design chemical dose changes or if the type of coagulant changes. Use the AguaClara equations that calculate pipe diameters.)

[/Monroe/]: # (The coagulant dosing system needs to be able to deliver the maximum dose required. I realize we don't have that info. Make up an assumption, state it as part of your basis of design and then proceed. If your design is done correctly a change to that input will be reflected immediately in your design recommendations.)

[/Monroe/]: # (Start thinking about how many coagulant feed systems will be needed based on the number of rapid mix units. Pull that information from fmt. see my example below.)

In [7]:
FeCl3_dose_max = 30 * u.mg/u.L # add this to our Basis of design in fmt.
FeCl3_stock = 400 * u.g/u.L # is this correct?
Q_FeCl3_max_dose = (fmt.q_max * FeCl3_dose_max / fmt.n_rm_mech / FeCl3_stock).to(u.mL/u.s)
print('The maximum FeCl3 dosing flow rate is',Q_FeCl3_max_dose)

The maximum FeCl3 dosing flow rate is 382.3 milliliter / second


In [0]:
#volume FeCl3 for 30 days via dosage 
time = 30 * u.days
density_fecl3 = 1.4*u.g/(u.mL) # for 40% m/v FeCl3 
mu_fecl3 = 0.12 * u.N * u.s / u.m**2
nu_fecl3 = mu_fecl3/density_fecl3
#print("Kinematic viscosity (nu) of 40% FeCl3 is", nu_fecl3.to(u.m**2/u.s))
tank_volume = fmt.Fe_c_ave /density_fecl3* time * fmt.q_ave
fecl3_month_mass = tank_volume*density_fecl3
print ("The volume of FeCl3 needed for 30 days based on the recommended dosage is",tank_volume.to(u.kgal))
print("The mass of FeCl3 needed for 30 days based on the recommended dosage is", fecl3_month_mass.to(u.klbs))

#volume FeCl3 for 30 days via annual mass estimate
fecl3_ann_est = 4142 * u.tons
fecl3_month = fecl3_ann_est *u.month/u.year 
fecl3_month_vol = fecl3_month /density_fecl3
print("The volume of FeCl3 needed for 30 days based on the SOW annual estimate is", fecl3_month_vol.to(u.kgal))

#cost of FeCl3 
unit_cost_fecl3 = 0.2243 * u.USD/u.lbs
ann_cost_fecl3 = unit_cost_fecl3*fecl3_month_mass* 1*u.year / time
print('The annual cost of FeCl3 is', ann_cost_fecl3.to(u.USD))

#volume of polymer for 30 days via annual mass estimate
polymer_ann_est = 165 * u.tons
polymer_month = polymer_ann_est *u.month/u.year 
print("The mass of polymer needed for 30 days based on the SOW annual estimate is", polymer_month.to(u.ton))

In [0]:
#rough estimates of parameters
elevation_tank = 0 *u.feet
elevation_rapid_mix = 2 *u.feet
pipe_d = 3*u.inches
pipe_l = 1000 *u.feet
stock_fecl_c = 4.4 *u.lb/u.gal
num_elbow = 4
k_elbow = .3 #Frank Fluid Mechanics pg 391, assumed 90 deg. regular flanged 4 in. nominal diameter
temp = 20 *u.degC
e = .0015 *u.mm #Frank Fluid Mechanics pg 371, assumed plastic

#calculating power pump 
flow_coagulant = fmt.Fe_c_ave * fmt.q_ave / stock_fecl_c
velocity_coagulant = flow_coagulant /(np.pi/4*(pipe_d**2))
minor_loss = num_elbow * k_elbow * (velocity_coagulant**2)/(2*ac.GRAVITY)
major_loss = ac.headloss_major_pipe(flow_coagulant, pipe_d, pipe_l, ac.viscosity_kinematic_water(temp), e)
head_loss = minor_loss + major_loss
head_pump = head_loss + elevation_rapid_mix - elevation_tank
power_pump = flow_coagulant*density_fecl3*ac.GRAVITY*head_pump

print("The power required to pump the coagulant to the treatment train is",power_pump.to(u.watt))

In [0]:
# Hering & Kneebone 2002 (p.169)
fecl3_dose_avg_CTP = 5.7 * u.mg/u.L
polymer_dose_avg_CTP = 1.7 * u.mg/u.L
# Assume that maximum dose is double
fecl3_dose_max = fecl3_dose_avg_CTP * 2

fecl3_strength = 0.4 # Active strength of ferric chloride solution
fecl3_mass_delivery_rate = (fmt.q_max * fecl3_dose_max)/fecl3_strength
fecl3_vol_delivery_rate = fecl3_mass_delivery_rate/density_fecl3
print("The maximum feed rate of ferric chloride solution is", fecl3_vol_delivery_rate.to(u.gal/u.hour).)

As the maximum feed rate is 394.7 gallons per hour, we will need to use a positive displacement diaphragm pump.